In [6]:
import requests
import pandas as pd
import re

def extract_album_id(album_link):
    match = re.search(r'/album/.*?/([\w-]+)\.html', album_link or "")
    return match.group(1) if match else "N/A"

def determine_album_type(track_count):
    if track_count == 1:
        return "Single"
    elif 2 <= track_count <= 6:
        return "EP"
    else:
        return "Regular"

def fetch_artist_songs(artist_name):
    url = f"http://localhost:5000/api/artistsongs?name={artist_name}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        songs = data.get("songs", [])
        
        records = []
        album_tracks = {}
        
        for song in songs:
            album_id = extract_album_id(song.get("albumLink", "") or "")
            album_name = song.get("album", song.get("title", "N/A"))
            tracklist = song.get("tracklist", [])
            
            if not tracklist:
                tracklist = [{"title": song.get("title", "N/A"), "link": song.get("link", "N/A")}]  # Nếu không có tracklist, tạo dữ liệu từ bài hát chính
            
            if album_id not in album_tracks:
                album_tracks[album_id] = len(tracklist)
            
            for track in tracklist:
                records.append({
                    "album_id": album_id,
                    "album_name": album_name,
                    "tracklist": track.get("title", "N/A"),
                    "release_date": song.get("releaseDate", "Unknown"),
                    "provided_by": song.get("providedBy", "Unknown"),
                    "featured_artists": song.get("featuredArtists", "Unknown"),
                    "album_artist": song.get("albumOwner", "Unknown"),
                    "ZingMP3": track.get("link", "N/A")
                })
        
        df = pd.DataFrame(records)
        df = df.drop_duplicates(subset=["album_id", "album_name", "tracklist"])
        df = df.sort_values(by=["album_name", "tracklist"], ascending=[True, True])
        
        # Thêm cột phân loại album
        df["album_type"] = df["album_id"].map(lambda x: determine_album_type(album_tracks.get(x, 1)))
        
        return df
    else:
        print("Error fetching data:", response.status_code)
        return None

artist_name = 'Rapper-Ngan'
df = fetch_artist_songs(artist_name)



In [7]:
df.to_excel(f'{artist_name}_songZingMP3.xlsx')
df.head()

,album_id,album_name,tracklist,release_date,provided_by,featured_artists,album_artist,ZingMP3,album_type
32,60BODZB8,9X (Single),9X,04/05/2020,MIXUS,"DLblack, Ngắn, Mêk Team","DLblack, Ngắn, Mêk Team",https://zingmp3.vn/bai-hat/9X-DLblack-Ngan-Mek...,Single
22,Z6DU0B8O,9X (Single),Trở Về,2019,MIXUS,"Ngắn, Hải Phòng Sound","Ngắn, Hải Phòng Sound",https://zingmp3.vn/bai-hat/Tro-Ve-Ngan-Hai-Pho...,Single
43,6909U6OC,Bình Dân (Remix Version) (Single),Bình Dân (Remix Version),10/09/2021,MIXUS,"Ngắn, Monkieq","Ngắn, Monkieq",https://zingmp3.vn/bai-hat/Binh-Dan-Remix-Vers...,Single
56,6BUBDBAI,Bụi Thiên Thần (EP),Cuộc Đời Này Ngắn,22/07/2022,LOOPS Music,Ngắn,Ngắn,https://zingmp3.vn/bai-hat/Cuoc-Doi-Nay-Ngan-N...,EP
60,6BUBDBAI,Bụi Thiên Thần (EP),Người Nghệ Sĩ Cô Đơn,22/07/2022,LOOPS Music,Ngắn,Ngắn,https://zingmp3.vn/bai-hat/Nguoi-Nghe-Si-Co-Do...,EP


In [13]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

# Hàm xác định loại album
def determine_album_type(track_count):
    if track_count <= 3:
        return "Single"
    elif 4 <= track_count <= 6:
        return "EP"
    else:
        return "Regular"

# Hàm lấy tất cả bài hát của nghệ sĩ
def get_artist_tracks_all(artist_name):
    client_id = "c7e1fe3ffe674920a01f9b016e6ae5df"
    client_secret = "215c6808dea74656b3629b306182ac4b"
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

    # 🔎 Tìm nghệ sĩ theo tên
    results = sp.search(q=artist_name, type='artist', limit=1)
    if not results['artists']['items']:
        print("Không tìm thấy nghệ sĩ.")
        return pd.DataFrame()

    artist_id = results['artists']['items'][0]['id']

    track_data = []
    seen_tracks = set()
    album_tracks_count = {}


    offset = 0
    while True:
        albums = sp.artist_albums(artist_id, album_type='album,single', limit=50, offset=offset)
        if not albums['items']:
            break
        for album in albums['items']:
            album_id = album['id']
            album_name = album['name']
            album_release_date = album.get('release_date', 'Unknown')
            album_owner = album['artists'][0]['name']

            album_info = sp.album(album_id)
            label = album_info.get('label', 'Unknown')

           
            tracks = sp.album_tracks(album_id)['items']
            album_tracks_count[album_name] = len(tracks)  # Ghi lại số lượng bài hát

            for track in tracks:
                track_id = track['id']
                if track_id in seen_tracks:
                    continue
                seen_tracks.add(track_id)

                track_title = track['name']
                link_spotify = track['external_urls']['spotify']
                featured_artists = [artist['name'] for artist in track['artists'] if artist['id'] != artist_id]
                featured_artists = ", ".join(featured_artists) if featured_artists else "None"

                # Xử lý ngày phát hành
                try:
                    album_release_date = pd.to_datetime(album_release_date, errors='coerce').strftime("%d/%m/%Y")
                except:
                    album_release_date = "Unknown"

                track_data.append([album_name, track_title, album_release_date, featured_artists, album_owner, label, link_spotify])
        offset += 50  

   
    top_tracks = sp.artist_top_tracks(artist_id, country="US")['tracks']
    for track in top_tracks:
        track_id = track['id']
        if track_id in seen_tracks:
            continue
        seen_tracks.add(track_id)

        track_title = track['name']
        link_spotify = track['external_urls']['spotify']
        album = track['album']
        album_name = album['name']
        album_release_date = album.get('release_date', 'Unknown')
        album_owner = album['artists'][0]['name']
        featured_artists = [artist['name'] for artist in track['artists'] if artist['id'] != artist_id]
        featured_artists = ", ".join(featured_artists) if featured_artists else "None"

        # 🔹 Lấy thông tin label của album
        album_info = sp.album(album['id'])
        label = album_info.get('label', 'Unknown')

        try:
            album_release_date = pd.to_datetime(album_release_date, errors='coerce').strftime("%d/%m/%Y")
        except:
            album_release_date = "Unknown"

        track_data.append([album_name, track_title, album_release_date, featured_artists, album_owner, label, link_spotify])
        album_tracks_count[album_name] = album_tracks_count.get(album_name, 0) + 1  # Cập nhật số bài hát


    columns = ["album_name", "tracklist", "release_date", "featured_artists", "album_owner_", "provided_by", "Link_Spotify"]
    df = pd.DataFrame(track_data, columns=columns)

    df['album_type'] = df['album_name'].map(lambda x: determine_album_type(album_tracks_count.get(x, 0)))
    df = df.sort_values(by=["album_name", "tracklist"], ascending=[True, True])
 

    return df

# Chạy chương trình
if __name__ == "__main__":
    artist_name_Spotify = 'Ngắn'
    df_tracks = get_artist_tracks_all(artist_name_Spotify)



    

C:\Users\user\AppData\Local\Temp\ipykernel_19492\201906038.py:64: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  album_release_date = pd.to_datetime(album_release_date, errors='coerce').strftime("%d/%m/%Y")


In [14]:

df_tracks.to_excel(f"{artist_name_Spotify}_songSpotify.xlsx", index=False)
df_tracks.head()

,album_name,tracklist,release_date,featured_artists,album_owner_,provided_by,Link_Spotify,album_type
24,120Bpm (ft. Long Mộng Gà),120Bpm (ft. Long Mộng Gà),10/03/2023,Antoneus Maximus,Ngắn,MaiDao Music,https://open.spotify.com/track/3QSw9fOs1Qe14tg...,Single
62,9X,9X,04/05/2020,"DLBlack, Mêk Team",DLBlack,Zing MP3,https://open.spotify.com/track/2Gj8BVWfiqHTykY...,Single
50,Bình Dân,Bình Dân,03/09/2021,None,Ngắn,Yin Yang Media,https://open.spotify.com/track/3dhDxLqZScUdYDx...,Single
51,Bình Dân,Bình Dân - Beat,09/03/2021,None,Ngắn,Yin Yang Media,https://open.spotify.com/track/0eof9V6sNRlPtiQ...,Single
30,Bụi Thiên Thần,Cuộc Đời Này Ngắn,27/07/2022,None,Ngắn,LOOPS Music,https://open.spotify.com/track/3UmZvOmW9QM7W1F...,EP


In [22]:
import pandas as pd
import re
from fuzzywuzzy import process

def find_best_match(title, choices, threshold=90):
    match, score = process.extractOne(title, choices)
    return match if score >= threshold else None

# Tạo từ điển album từ df
album_titles_dict = {title: title for title in df["album_name"]}

# Tìm album phù hợp nhất giữa df_tracks và df
df_tracks["best_match_album"] = df_tracks["album_name"].apply(
    lambda x: find_best_match(x, album_titles_dict.keys())
)

# Gộp dữ liệu dựa trên album_name khớp được
df_merged = pd.merge(df_tracks, df, left_on="best_match_album", right_on="album_name", how="outer", suffixes=("_Spotify", "_ZingMP3"))

# Xử lý nếu không có dữ liệu trên Spotify thì lấy từ ZingMP3
df_merged = df_merged.assign(
    album_name=df_merged["album_name_Spotify"].fillna(df_merged["album_name_ZingMP3"]),
    album_type=df_merged["album_type_Spotify"].fillna(df_merged["album_type_ZingMP3"]),
    track_title=df_merged["tracklist_Spotify"].fillna(df_merged["tracklist_ZingMP3"]),
    #featured_artists=df_merged["featured_artists_Spotify"].fillna(df_merged["featured_artists"]),
    #album_owner=df_merged["album_owner_Spotify"].fillna(df_merged["album_owner"]),
    #release_date=df_merged["release_date_Spotify"].fillna(df_merged["release_date"]),
    #label=df_merged["provided_by_Spotify"].fillna(df_merged["provided_by"])
)

# Xóa cột không cần thiết
df_ = df_merged.drop(columns=["best_match_album", "album_name_Spotify", "album_name_ZingMP3"])

# Đổi tên cột
df_ = df_.rename(columns={
    "album_name": "album_name",
    "track_title": "tracklist(danh sách bài hát)",
    "featured_artists": "Song artist(nghệ sĩ tham gia bài hát)",
    "album_owner": "Album artist (nghệ sĩ sở hữu album)",
    "release_date": "Ngày phát hành",
    "label": "Cung cấp bởi",
    "linkzingmp3": "ZingMP3",
    "Link_Spotify": "Spotify",
    "album_type": "album_type",
    "album_id": "Mã định danh album"
})

# Lọc bỏ trùng lặp theo album_name và tracklist
df_ = df_.drop_duplicates(subset=["album_name", "tracklist(danh sách bài hát)"], keep="first")

# Chọn các cột cần thiết
desired_columns = [
    "album_name", 
    "Album artist (nghệ sĩ sở hữu album)", 
    "album_type",
    "tracklist(danh sách bài hát)", 
    "Ngày phát hành",
    "Song artist(nghệ sĩ tham gia bài hát)", 
    "Cung cấp bởi",
    "Mã định danh album",
    "ZingMP3",
    "Spotify"
]

df_ = df_[[col for col in desired_columns if col in df_.columns]]
df_ = df_.sort_values(by=[ "album_name","Mã định danh album"], ascending=[True, True])

# Xuất file Excel
df_.to_excel(f"{artist_name}_AlbumsZingMp3_Spot.xlsx", index=False)

df_.head()




,album_name,album_type,tracklist(danh sách bài hát),Mã định danh album,ZingMP3,Spotify
102,120Bpm (ft. Long Mộng Gà),Single,120Bpm (ft. Long Mộng Gà),NaN,NaN,https://open.spotify.com/track/3QSw9fOs1Qe14tg...
0,9X,Single,9X,60BODZB8,https://zingmp3.vn/bai-hat/9X-DLblack-Ngan-Mek...,https://open.spotify.com/track/2Gj8BVWfiqHTykY...
2,Bình Dân,Single,Bình Dân,6909U6OC,https://zingmp3.vn/bai-hat/Binh-Dan-Remix-Vers...,https://open.spotify.com/track/3dhDxLqZScUdYDx...
3,Bình Dân,Single,Bình Dân - Beat,6909U6OC,https://zingmp3.vn/bai-hat/Binh-Dan-Remix-Vers...,https://open.spotify.com/track/0eof9V6sNRlPtiQ...
4,Bụi Thiên Thần,EP,Cuộc Đời Này Ngắn,6BUBDBAI,https://zingmp3.vn/bai-hat/Cuoc-Doi-Nay-Ngan-N...,https://open.spotify.com/track/3UmZvOmW9QM7W1F...


In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

def search_artist_nhaccuatui(driver, artist_name):
    """Tìm nghệ sĩ trên NhacCuaTui và lấy link trang nghệ sĩ."""
    driver.get("https://www.nhaccuatui.com/")
    time.sleep(3)

    try:
        search_box = driver.find_element(By.CSS_SELECTOR, "input#txtSearch")
        search_box.click()
        time.sleep(1)

        search_box.send_keys(artist_name)
        time.sleep(3)

        suggestions = driver.find_elements(By.CSS_SELECTOR, ".info-search .qsItem a")
        for suggestion in suggestions:
            if "nghe-si" in suggestion.get_attribute("href"):
                return suggestion.get_attribute("href")
    except Exception as e:
        print(f"Lỗi khi tìm nghệ sĩ {artist_name}: {e}")
    return None

def get_artist_songs(driver, artist_song_url):
    """Lấy danh sách bài hát của nghệ sĩ từ tất cả các trang."""
    driver.get(artist_song_url)
    time.sleep(3)
    song_list = []
    while True:
        song_blocks = driver.find_elements(By.CSS_SELECTOR, ".box-content-music-list .info_song")
        for song_block in song_blocks:
            try:
                song_name = song_block.find_element(By.CSS_SELECTOR, "h3 a").text
                song_link = song_block.find_element(By.CSS_SELECTOR, "h3 a").get_attribute("href")
                song_list.append(("N/A", song_name, song_link))
            except Exception as e:
                print(f"Lỗi khi lấy bài hát: {e}")
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.number[rel='next']")
            next_link = next_button.get_attribute("href")
            if next_link:
                driver.get(next_link)
                time.sleep(3)
            else:
                break
        except:
            break
    return song_list

def get_artist_albums(driver, artist_album_url):
    """Lấy danh sách Album của nghệ sĩ."""
    driver.get(artist_album_url)
    time.sleep(3)

    album_list = []
    while True:
        album_blocks = driver.find_elements(By.CSS_SELECTOR, ".box-left-album a.box_absolute")
        for album in album_blocks:
            try:
                album_link = album.get_attribute("href")
                album_list.append(album_link)
            except Exception as e:
                print(f"Lỗi khi lấy album: {e}")
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.number[rel='next']")
            next_link = next_button.get_attribute("href")
            if next_link:
                driver.get(next_link)
                time.sleep(3)
            else:
                break
        except:
            break
    return album_list

def get_album_songs(driver, album_url):
    """Lấy danh sách bài hát trong Album."""
    driver.get(album_url)
    time.sleep(3)
    try:
        album_name = driver.find_element(By.CSS_SELECTOR, ".name_title").text
        song_elements = driver.find_elements(By.CSS_SELECTOR, "li[id^='itemSong_']")
        album_songs = [(album_name,
                        s.find_element(By.CSS_SELECTOR, "meta[itemprop='name']").get_attribute("content"),
                        s.find_element(By.CSS_SELECTOR, "meta[itemprop='url']").get_attribute("content"))
                        for s in song_elements]
        return album_songs
    except Exception as e:
        print(f"Lỗi khi lấy bài hát từ album {album_url}: {e}")
    return []

driver = webdriver.Chrome()
artist = "Ngắn"

artist_page = search_artist_nhaccuatui(driver, artist)
if artist_page:
    print(f"Trang nghệ sĩ: {artist_page}")

    artist_song_page = artist_page.replace(".html", ".bai-hat.html")
    artist_album_page = artist_page.replace(".html", ".playlist.html")

    artist_songs = get_artist_songs(driver, artist_song_page)
    print(pd.DataFrame(artist_songs, columns=["Album", "Tên bài hát", "Link bài hát"]))
    
    albums = get_artist_albums(driver, artist_album_page)
    print(f"Tìm thấy {len(albums)} album.")
    artist_songs = [(song_name, song_name, song_link) for _, song_name, song_link in artist_songs]
    all_songs = artist_songs[:]
    for album_link in albums:
        album_songs = get_album_songs(driver, album_link)
        all_songs.extend(album_songs)

    df_nct = pd.DataFrame(all_songs, columns=["album_name", "tracklist(danh sách bài hát)", "Link bài hát"])
    df_nct.to_excel(f"{artist}_NhacCuaTui_Songs.xlsx", index=False)
    print(f"Lấy được {len(all_songs)} bài hát của {artist}, lưu vào file Excel!")
else:
    print("Không tìm thấy nghệ sĩ.")
driver.quit()


Trang nghệ sĩ: https://www.nhaccuatui.com/nghe-si-ngan.html
   Album                                        Tên bài hát  \
0    N/A  NGẮN | Trăng Tròn - phiên bản quẩy tung nóc đê...   
1    N/A           TÌM HÀNH TINH KHÁC | VŨ CÁT TƯỜNG x NGẮN   
2    N/A                      NGẮN | BÌNH DÂN | MV Official   
3    N/A  Em Ở Nơi Đâu - Ngắn aka Lil Shadow Hải Phòng S...   
4    N/A  NGẮN | NƠI TAO SINH RA | MV OFFICIAL | Prod - ...   
..   ...                                                ...   
74   N/A          Sống Cho Hết Đời Thanh Xuân 4 (Chill Ver)   
75   N/A                            Cạm Bẫy (The Rap Game )   
76   N/A                                            Cạm Bẫy   
77   N/A                                  Cypher The Heroes   
78   N/A                                   Tìm Đường Về Nhà   

                                         Link bài hát  
0   https://www.nhaccuatui.com/bai-hat/ngan-trang-...  
1   https://www.nhaccuatui.com/bai-hat/tim-hanh-ti...  
2   https://www

In [7]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import time
# import pandas as pd

# def search_nhaccuatui(artist_name):
#     driver = webdriver.Chrome()
    
#     search_url_songs = f"https://www.nhaccuatui.com/tim-kiem/bai-hat?q={artist_name}&b=keyword&l=tat-ca&s=default"
#     search_url_playlists = f"https://www.nhaccuatui.com/tim-kiem/playlist?q={artist_name}&b=keyword&l=tat-ca&s=default"
    
#     driver.get(search_url_songs)
#     time.sleep(3)
    
#     song_list = []
    
#     while True:
#         songs = driver.find_elements(By.CSS_SELECTOR, ".box_info h3.title_song a")
#         artists = driver.find_elements(By.CSS_SELECTOR, ".box_info h4.singer_song")
        
#         for song, artist in zip(songs, artists):
#             artist_names = ", ".join([a.text for a in artist.find_elements(By.TAG_NAME, "a")])
#             song_list.append((song.text, song.get_attribute("href"), artist_names))
        
#         try:
#             next_button = driver.find_element(By.CSS_SELECTOR, "a.number[rel='next']")
#             driver.execute_script("arguments[0].click();", next_button)
#             time.sleep(3)
#         except:
#             break
    
#     driver.get(search_url_playlists)
#     time.sleep(3)
    
#     checked_playlists = set()
#     playlist_links = []
    
#     while True:
#         try:
#             playlists = driver.find_elements(By.CSS_SELECTOR, ".box_info h3.title_song a")
#             for p in playlists:
#                 link = p.get_attribute("href")
#                 if link and link not in checked_playlists:
#                     playlist_links.append(link)
#                     checked_playlists.add(link)
#         except Exception as e:
#             print(f"Lỗi khi lấy danh sách playlist: {e}")
        
#         try:
#             next_button = driver.find_element(By.CSS_SELECTOR, "a.number[rel='next']")
#             driver.execute_script("arguments[0].click();", next_button)
#             time.sleep(3)
#         except:
#             break
    
#     playlist_list = []
    
#     for link in playlist_links:
#         retry = 3
#         while retry > 0:
#             try:
#                 driver.get(link)
#                 time.sleep(3)
                
#                 album_name = driver.find_element(By.CSS_SELECTOR, ".name_title").text
                
#                 song_elements = driver.find_elements(By.CSS_SELECTOR, "li[id^='itemSong_']")
#                 playlist_songs = [(s.find_element(By.CSS_SELECTOR, "meta[itemprop='name']").get_attribute("content"),
#                                    s.find_element(By.CSS_SELECTOR, "meta[itemprop='url']").get_attribute("content"))
#                                    for s in song_elements]
                
#                 playlist_list.append((album_name, playlist_songs, link))
#                 break
#             except Exception as e:
#                 print(f"Lỗi khi lấy dữ liệu playlist {link}: {e}")
#                 driver.refresh()
#                 time.sleep(3)
#                 retry -= 1
    
#     driver.quit()
#     return song_list, playlist_list

# if __name__ == "__main__":
#     artist = "MCK"
#     songs, playlists = search_nhaccuatui(artist)
    
#     data = []
    
#     for song_title, song_link, _ in songs:
#         data.append([song_title, song_title, song_link])
    
#     for album_name, playlist_songs, link in playlists:
#         for title, song_link in playlist_songs:
#             data.append([album_name, title, song_link])
    
#     df_nct = pd.DataFrame(data, columns=["album_name", "tracklist(danh sách bài hát)", "Nhaccuatui"])
    
#     # Lưu vào file CSV
#     df_nct.to_excel(f"{artist}_NhacCuaTui.xlsx", index=False)
    
                


In [6]:
# import pandas as pd
# from fuzzywuzzy import process

# # Hàm fuzzy matching để chuẩn hóa dữ liệu
# def fuzzy_standardize(value, reference_values, threshold=95):
#     if not value or pd.isna(value):
#         return ""
#     best_match = process.extractOne(value, reference_values, score_cutoff=threshold)
#     return best_match[0] if best_match else value

# # Kiểm tra nếu df_ và df_nct tồn tại

# if "album_name" in df_nct.columns and "album_name" in df_.columns:
#         unique_albums_nct = df_nct["album_name"].dropna().unique()
#         df_["album_name"] = df_["album_name"].apply(lambda x: fuzzy_standardize(x, unique_albums_nct))

# if "tracklist(danh sách bài hát)" in df_nct.columns and "tracklist(danh sách bài hát)" in df_.columns:
#         unique_titles_nct = df_nct["tracklist(danh sách bài hát)"].dropna().unique()
#         df_["final_title"] = df_["tracklist(danh sách bài hát)"].apply(lambda x: fuzzy_standardize(x, unique_titles_nct))

#     # Merge dữ liệu từ NCT vào df_
# df_merged_nct = pd.merge(df_, df_nct, left_on=["final_title", "album_name"], 
#                              right_on=["tracklist(danh sách bài hát)", "album_name"], 
#                              how="outer", suffixes=("", "_NCT"))

#     # Xử lý lỗi nếu cột "_NCT" không tồn tại
# if "album_name_NCT" in df_merged_nct.columns:
#         df_merged_nct["album_name"] = df_merged_nct["album_name"].fillna(df_merged_nct["album_name_NCT"])
    
# if "tracklist(danh sách bài hát)_NCT" in df_merged_nct.columns:
#         df_merged_nct["tracklist(danh sách bài hát)"] = df_merged_nct["tracklist(danh sách bài hát)"].fillna(df_merged_nct["tracklist(danh sách bài hát)_NCT"])

#     # Xóa cột không cần thiết nếu tồn tại
# columns_to_drop = ["album_name_NCT", "final_title", "tracklist(danh sách bài hát)_NCT"]
# df_final = df_merged_nct.drop(columns=[col for col in columns_to_drop if col in df_merged_nct.columns])

#     # Xử lý khoảng trắng nhưng giữ nguyên dấu
# df_final["album_name"] = df_final["album_name"].astype(str).str.strip().replace(r"\s+", " ", regex=True)
# df_final["tracklist(danh sách bài hát)"] = df_final["tracklist(danh sách bài hát)"].astype(str).str.strip().replace(r"\s+", " ", regex=True)

#     # Chuẩn hóa cột "Nhaccuatui"
# if "Link bài hát" in df_final.columns:
#         df_final.rename(columns={"Link bài hát": "Nhaccuatui"}, inplace=True)

# if "Nhaccuatui" in df_final.columns:
#         df_final["Nhaccuatui"] = df_final["Nhaccuatui"].astype(str).str.strip()

#     # Xóa trùng lặp
# df_final = df_final.drop_duplicates(subset=["album_name", "tracklist(danh sách bài hát)", "Nhaccuatui"])

#     # Sắp xếp giữ nguyên dấu
# df_final = df_final.sort_values(by=["album_name", "tracklist(danh sách bài hát)"], ascending=[True, True])

#     # Xuất ra file Excel
# df_final.to_excel("songSpot+ZingMP3+NCT.xlsx", index=False)


# df_final.head()


In [25]:
import pandas as pd
from fuzzywuzzy import process

# Hàm chuẩn hóa dữ liệu bằng fuzzy matching
def fuzzy_standardize(value, reference_values, threshold=85):
    if not value or pd.isna(value):
        return ""
    best_match = process.extractOne(value, reference_values, score_cutoff=threshold)
    return best_match[0] if best_match else value


# Chuẩn hóa album_name giữa các nguồn
album_names_all = pd.concat([df["album_name"], df_tracks["album_name"], df_nct["album_name"]]).dropna().unique()

df["album_name"] = df["album_name"].apply(lambda x: fuzzy_standardize(x, album_names_all))
df_tracks["album_name"] = df_tracks["album_name"].apply(lambda x: fuzzy_standardize(x, album_names_all))
df_nct["album_name"] = df_nct["album_name"].apply(lambda x: fuzzy_standardize(x, album_names_all))

# Đổi tên cột để đồng bộ
df_tracks = df_tracks.rename(columns={
    "tracklist": "track_title",
    "release_date_Spotify": "release_date",
    "featured_artists_Spotify": "featured_artists",
    "album_owner_Spotify": "album_owner",
    "provided_by_Spotify": "provided_by",
    "Link_Spotify": "link_spotify"
})

df_nct = df_nct.rename(columns={
    "tracklist(danh sách bài hát)": "track_title",
    "Link bài hát": "link_nhaccuatui"
})

# Hợp nhất dữ liệu từ ZingMP3 và Spotify
df_merged = pd.merge(df, df_tracks, on="album_name", how="outer", suffixes=("_ZingMP3", "_Spotify"))

# Xử lý ưu tiên album_name từ ZingMP3
df_merged["album_name"] = df_merged["album_name"].combine_first(df_merged["album_name"])

# Hợp nhất với dữ liệu từ NhacCuaTui
df_final = pd.merge(df_merged, df_nct, on="album_name", how="outer")

# Xử lý các cột đầu ra
df_final = df_final[[
    "album_name",
    "album_owner_Spotify" if "album_owner_Spotify" in df_final.columns else "album_owner_ZingMP3",
    "album_type_ZingMP3" if "album_type_ZingMP3" in df_final.columns else "album_type_Spotify",
    "track_title" if "track_title" in df_final.columns else "track_title_ZingMP3",
    "release_date_ZingMP3" if "release_date_ZingMP3" in df_final.columns else "release_date_Spotify",
    "featured_artists_ZingMP3" if "featured_artists_ZingMP3" in df_final.columns else "featured_artists_Spotify",
    "provided_by_ZingMP3" if "provided_by_ZingMP3" in df_final.columns else "provided_by_Spotify",
    "album_id",
    "linkzingmp3",
    "link_spotify",
    "link_nhaccuatui"
]]
df_final = df_final.drop_duplicates(subset=["album_name", "track_title"], keep="first")

# Xuất file Excel
df_final.to_excel("df_final.xlsx", index=False)
print("Gộp dữ liệu hoàn tất! Xuất ra df_final.xlsx.")


KeyError: "['album_owner_ZingMP3', 'track_title_ZingMP3', 'linkzingmp3'] not in index"

In [34]:
print(df_merged.columns)  # Kiểm tra tên cột trong df_tracks
print(df_final.columns)  # Kiểm tra tên cột trong df


Index(['album_id', 'album_name', 'track_title_ZingMP3', 'release_date_ZingMP3',
       'provided_by_ZingMP3', 'featured_artists_ZingMP3',
       'album_owner_ZingMP3', 'linkzingmp3', 'album_type_ZingMP3',
       'track_title_Spotify', 'release_date_Spotify',
       'featured_artists_Spotify', 'album_owner_Spotify',
       'provided_by_Spotify', 'link_spotify', 'album_type_Spotify',
       'best_match_album'],
      dtype='object')
Index(['album_id', 'album_name', 'track_title_ZingMP3', 'release_date_ZingMP3',
       'provided_by_ZingMP3', 'featured_artists_ZingMP3',
       'album_owner_ZingMP3', 'linkzingmp3', 'album_type_ZingMP3',
       'track_title_Spotify', 'release_date_Spotify',
       'featured_artists_Spotify', 'album_owner_Spotify',
       'provided_by_Spotify', 'link_spotify', 'album_type_Spotify',
       'best_match_album', 'track_title', 'link_nhaccuatui'],
      dtype='object')


In [15]:
print(df_merged.columns)



Index(['album_name', 'Album artist (nghệ sĩ sở hữu album)', 'album_type',
       'tracklist(danh sách bài hát)', 'Ngày phát hành',
       'Song artist(nghệ sĩ tham gia bài hát)', 'Cung cấp bởi',
       'Mã định danh album', 'ZingMP3', 'Spotify', 'final_title',
       'tracklist(danh sách bài hát)_NCT', 'Link bài hát'],
      dtype='object')


In [63]:
import pandas as pd
from fuzzywuzzy import process

# Hàm chuẩn hóa dữ liệu bằng fuzzy matching
def fuzzy_standardize(value, reference_values, threshold=85):
    if not value or pd.isna(value):
        return ""
    best_match = process.extractOne(value, reference_values, score_cutoff=threshold)
    return best_match[0] if best_match else value

# Giả sử bạn đã có các DataFrame: df (ZingMP3), df_tracks (Spotify), df_nct (NhacCuaTui)

# Bước 1: Lấy danh sách album chuẩn từ cả ba nguồn
all_album_names = pd.concat([df["album_name"], df_tracks["album_name"], df_nct["album_name"]]).dropna().unique()

# Chuẩn hóa tên album để tránh lỗi trùng lặp
df["album_name"] = df["album_name"].apply(lambda x: fuzzy_standardize(x, all_album_names))
df_tracks["album_name"] = df_tracks["album_name"].apply(lambda x: fuzzy_standardize(x, all_album_names))
df_nct["album_name"] = df_nct["album_name"].apply(lambda x: fuzzy_standardize(x, all_album_names))

# Bước 2: Gộp cả ba DataFrame cùng lúc
df_final = df.merge(df_tracks, on="album_name", how="outer", suffixes=('_ZingMP3', '_Spotify'))\
             .merge(df_nct, on="album_name", how="outer", suffixes=('', '_Nhaccuatui'))

# Bước 3: Điền dữ liệu ưu tiên theo thứ tự ZingMP3 → Spotify → NhacCuaTui
df_final["album_artist"] = df_final["album_artist_ZingMP3"].fillna(df_final["album_artist_Spotify"])
df_final["provided_by"] = df_final["provided_by_ZingMP3"].fillna(df_final["provided_by_Spotify"])
df_final["release_date"] = df_final["release_date_ZingMP3"].fillna(df_final["release_date_Spotify"])
df_final["song_artist"] = df_final["song_artist_ZingMP3"].fillna(df_final["song_artist_Spotify"])
df_final["album_type"] = df_final["album_type_ZingMP3"].fillna(df_final["album_type_Spotify"])


df_final.drop_duplicates(subset=["album_name", "track_title"], keep="first", inplace=True)
df_final
# # Bước 5: Chọn các cột quan trọng
# columns_needed = [
#     "album_name", "album_artist_Spotify", "album_artist_ZingMP3", "album_type",
#     "track_title", "release_date", "song_artist", "provided_by",
#     "ZingMP3", "Spotify", "Nhaccuatui"
# ]
# df_final = df_final[[col for col in columns_needed if col in df_final.columns]]

# # Xuất ra file Excel
df_final.to_excel("Album_Songs_Combined.xlsx", index=False)

# print("✅ Gộp dữ liệu hoàn tất! Cả ba nguồn đã được hợp nhất.")  



In [26]:
df_final = pd.merge(df_, df_nct, on=["album_name"], how="outer", suffixes=("", "_NCT"))

# Đảm bảo không bị mất cột từ df_
for col in df_.columns:
    if col not in df_final.columns:
        df_final[col] = df_[col]

# Xuất file Excel  
df_final = df_final.drop_duplicates(subset=["album_name", "tracklist(danh sách bài hát)"], keep="first")

# Xuất file Excel  
df_final.to_excel(f"{artist_name}_AlbumsZingMp3_Spot_NCT.xlsx", index=False)

df_final.head()


,album_name,album_type,tracklist(danh sách bài hát),Mã định danh album,ZingMP3,Spotify,track_title,link_nhaccuatui
0,120Bpm,NaN,NaN,NaN,NaN,NaN,120Bpm,https://www.nhaccuatui.com/bai-hat/120bpm-ngan...
1,120Bpm (ft. Long Mộng Gà),Single,120Bpm (ft. Long Mộng Gà),NaN,NaN,https://open.spotify.com/track/3QSw9fOs1Qe14tg...,NaN,NaN
2,9X,Single,9X,60BODZB8,https://zingmp3.vn/bai-hat/9X-DLblack-Ngan-Mek...,https://open.spotify.com/track/2Gj8BVWfiqHTykY...,NaN,NaN
3,Ai Cũng Có Một Thời,NaN,NaN,NaN,NaN,NaN,Ai Cũng Có Một Thời,https://www.nhaccuatui.com/bai-hat/ai-cung-co-...
4,Bình Dân,Single,Bình Dân,6909U6OC,https://zingmp3.vn/bai-hat/Binh-Dan-Remix-Vers...,https://open.spotify.com/track/3dhDxLqZScUdYDx...,Bình Dân,https://www.nhaccuatui.com/bai-hat/binh-dan-ng...


In [65]:
print(df_final.columns)


Index(['album_name_ZingSpotify', 'Album artist (nghệ sĩ sở hữu album)',
       'album_type', 'tracklist(danh sách bài hát)', 'Ngày phát hành',
       'Song artist(nghệ sĩ tham gia bài hát)', 'Cung cấp bởi',
       'Mã định danh album', 'ZingMP3', 'Spotify', 'final_title',
       'best_match_album_nct', 'album_name_NCT', 'track_title',
       'link_nhaccuatui'],
      dtype='object')
